# CNN 샘플 분석하기
## CNN 샘플에 필요한 라이브러리 임포트

In [1]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

device = 'cuda' if torch.cuda.is_available() else 'cpu'

## 랜덤 시드 고정 및 데이터셋 불러오기

In [2]:
# 랜덤 시드 고정
torch.manual_seed(0)

# GPU 사용 가능일 경우 랜덤 시드 고정
if device == 'cuda':
    torch.cuda.manual_seed_all(0)

# 데이터셋 가져오기
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True, 
                          transform=transforms.ToTensor(), 
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST\raw\train-images-idx3-ubyte.gz to MNIST_data/MNIST\raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST\raw\train-labels-idx1-ubyte.gz to MNIST_data/MNIST\raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST\raw\t10k-images-idx3-ubyte.gz to MNIST_data/MNIST\raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST\raw\t10k-labels-idx1-ubyte.gz to MNIST_data/MNIST\raw



## 하이퍼 파라미터 설정

In [3]:
# 하이퍼 파라미터 설정
learning_rate = 0.001
training_epochs = 15
batch_size = 100

# 데이터 로더 정의
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [4]:
# CNN 모델 정의
class CNN(torch.nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # 첫번째층
        # ImgIn shape=(?, 28, 28, 1)
        #    Conv     -> (?, 28, 28, 32)
        #    Pool     -> (?, 14, 14, 32)
        
        self.layer1 = torch.nn.Sequential(
            # 1채널의 이미지를 인풋으로 받는다.
            # 커널의 개수는 32개이며 사이즈는 3x3이다. 
            # 이미지를 계산할 때는 한 칸씩이동하여 계산하며
            # 패딩 값이 있으므로 이미지 사이즈가 줄지 않는다.
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            # 0이하의 값은 0이되며 0보다 큰 값만 살아 남는다.
            torch.nn.ReLU(),
            # 2x2마다 최대 값을 추출한다.
            # 이미지의 사이즈가 절반으로 줄어든다.
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        # 두번째층
        # ImgIn shape=(?, 14, 14, 32)
        #    Conv      ->(?, 14, 14, 64)
        #    Pool      ->(?, 7, 7, 64)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        # 전결합층 7x7x64 inputs -> 10 outputs
        self.fc = torch.nn.Linear(7 * 7 * 64, 10, bias=True)

        # 전결합층 한정으로 가중치 초기화
        # Sigmoid와 같은 S자 함수의 경우, 가장 중요한 것은 출력값들이 표준 정규 분포 형태를 갖게 하는 것
        # Xavier(사비에르) Initialization 방법은, 단순히 가중치를 작은 값의 표준편차를 갖는 형태로 초기화 하는 것이 아닌, 보다 발전된 방법
        # 출처: https://wooono.tistory.com/223
        torch.nn.init.xavier_uniform_(self.fc.weight)


    def forward(self, x):
        # 포워드 패스
        out = self.layer1(x)   # conv layer
        out = self.layer2(out) # conv layer
        out = out.view(out.size(0), -1)   # 전결합층을 위해서 Flatten
        out = self.fc(out) # full connected layer
        return out

    
# CNN 모델 생성
model = CNN().to(device)

# 로스와 최적화 함수 설정
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
'''
Adam은 GradientDescentOptimizer에 비해 몇 가지 장점을 제공
moving averages of the parameters를 사용(momentum) 
Adam은 더 큰 효과적인 step size를 사용할 수 있으며 알고리즘은 튜닝 없이 이 단계 크기로 수렴
Adam이 각 트레이닝 단계에서 각 매개 변수에 대해 더 많은 계산을 수행 
(이동 평균 및 분산을 유지하고 크기 조정 된 그라디언트를 계산하기 위해)
각 매개 변수에 대해 더 많은 상태를 유지 
(각 매개 변수의 평균 및 분산을 저장하기 위해 모델의 크기를 약 3 배로 늘림)
'''


'\nAdam은 GradientDescentOptimizer에 비해 몇 가지 장점을 제공\nmoving averages of the parameters를 사용(momentum) \nAdam은 더 큰 효과적인 step size를 사용할 수 있으며 알고리즘은 튜닝 없이 이 단계 크기로 수렴\nAdam이 각 트레이닝 단계에서 각 매개 변수에 대해 더 많은 계산을 수행 \n(이동 평균 및 분산을 유지하고 크기 조정 된 그라디언트를 계산하기 위해)\n각 매개 변수에 대해 더 많은 상태를 유지 \n(각 매개 변수의 평균 및 분산을 저장하기 위해 모델의 크기를 약 3 배로 늘림)\n'

In [5]:

# 데이터 로드
total_batch = len(data_loader)
print('총 배치의 수 : {}'.format(total_batch))

# 트레이닝 데이터셋으로 epoch 실행
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader: # 미니 배치 단위로 꺼내온다. X는 미니 배치, Y는 레이블.
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad() # 기울기 초기화
        hypothesis = model(X) # 모델 연산 후 결과 저장
        cost = criterion(hypothesis, Y) # 오류 산출
        cost.backward()  # 역전파 수행
        optimizer.step() # 업데이트 수행

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

    
# 학습을 진행하지 않을 것이므로 torch.no_grad()
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

총 배치의 수 : 600
[Epoch:    1] cost = 0.215979591
[Epoch:    2] cost = 0.0623616278
[Epoch:    3] cost = 0.0443787314
[Epoch:    4] cost = 0.0355432183
[Epoch:    5] cost = 0.0289978608
[Epoch:    6] cost = 0.0254101083
[Epoch:    7] cost = 0.0199933145
[Epoch:    8] cost = 0.0177171528
[Epoch:    9] cost = 0.0151013359
[Epoch:   10] cost = 0.0122582028
[Epoch:   11] cost = 0.0104048718
[Epoch:   12] cost = 0.00806191936
[Epoch:   13] cost = 0.00765087875
[Epoch:   14] cost = 0.00629990129
[Epoch:   15] cost = 0.00545077305


C:\Users\isc03\anaconda3\lib\site-packages\torchvision\datasets\mnist.py:80: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
C:\Users\isc03\anaconda3\lib\site-packages\torchvision\datasets\mnist.py:70: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy: 0.988099992275238
